<a href="https://colab.research.google.com/github/YannTyr/SSTV/blob/main/CCTV_SPb_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Number of SSTV cameras per district in Saint-Petersburg

## Data processing

In [ ]:
import geopandas as gpd
import pandas as pd

pd.set_option('display.max_columns', None)

In [ ]:
def load_cctv_data(cctv_link):
    cctv_data = gpd.read_file(cctv_link)
    cctv_data = cctv_data.rename(
        columns={
                "Порядковый номер": "N",
                "Район": "District",
                "Адрес": "Address",
                "Количество видеокамер": "Number_of_CCTV"
        })

    # Unify District data
    if cctv_data["District"][0][-6:] != " район":
        cctv_data["District"] += " район"

    # Convert numbers from string to integer
    cctv_data = cctv_data.astype({"Number_of_CCTV": 'float64'})
    cctv_data = cctv_data.astype({"Number_of_CCTV": 'int64'})
    return cctv_data


def aggregate_cctv(cctv_data):
    # Sum up all CCTV in the same districts
    cctv = cctv_data.groupby("District").agg(Number_of_CCTV=("Number_of_CCTV", "sum"))
    cctv = cctv.reset_index()
    return cctv

In [ ]:
def prepare_geometry(geometry_link):
    # Load districts' borders for the country
    russia_geometry = gpd.read_file(geometry_link, encoding='cp1251')

    # Extract districts for the city
    spb_geometry = russia_geometry.loc[russia_geometry.addr_regio=="Санкт-Петербург"]
    spb_geometry = spb_geometry.sort_values(by="name").reset_index()

    # Select important columns
    spb_geometry = spb_geometry.loc[:, ["name", "geometry"]]

    # Rename first column to join tables
    spb_geometry = spb_geometry.rename(columns={"name":"District"})
    return spb_geometry


def add_geometry(cctv, spb_geometry):
    # Join tables with geometry and number of CCTV
    districts = cctv.set_index("District").join(spb_geometry.set_index("District"))
    districts = districts.reset_index()

    # Conver to gpd formmat with 'WGS 84 / UTM zone 36N' projection
    districts_gdf = gpd.GeoDataFrame(districts, geometry=districts.geometry)
    districts_gdf = districts_gdf.to_crs({'init': 'epsg:32636'})

    # Calculate districts' 'area' and add such column
    # districts_gdf["area"] = districts_gdf["geometry"].area

    return districts_gdf


def add_difference(gdf_v):
    for v in versions:
        if v != "1":
            gdf_v[v]["Increase"] = gdf_v[v]["Number_of_CCTV"] - gdf_v[prev_v]["Number_of_CCTV"]
            gdf_v[v]["Increase_str"] = "+" + gdf_v[v]["Increase"].astype(str)
        else:
            gdf_v[v]["Increase"] = None
            gdf_v[v]["Increase_str"] = ""
        prev_v = v

In [ ]:
# Create borders of districts
geometry_link = "/content/drive/MyDrive/Colab Notebooks/CCTV_SPb/Адм-территориальные границы РФ (SHP, level_5)/admin_level_5.shp"
spb_geometry = prepare_geometry(geometry_link)

# Load and process different versions of cctv data (for last 4 years)
versions = {}
for v in [1, 5, 10, 13]:
    link = f"/content/drive/MyDrive/Colab Notebooks/CCTV_SPb/v{v}_Addresses_camera_installation.csv"
    cctv_data = load_cctv_data(link)  # crab table with camera addresses
    cctv = aggregate_cctv(cctv_data)  # calculate number of cameras per district
    districts_gdf = add_geometry(cctv, spb_geometry)  # add geometry data
    versions[str(v)] = districts_gdf  # save with other versions

# Add column with annual increase
add_difference(versions)

/usr/local/lib/python3.10/dist-packages/pyproj/crs/crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
/usr/local/lib/python3.10/dist-packages/pyproj/crs/crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
/usr/local/lib/python3.10/dist-packages/pyproj/crs/crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of 

### Check the results

In [ ]:
versions['13']

In [ ]:
spb_geometry.plot()

In [ ]:
spb_geometry

## Visualization

In [ ]:
import folium
import branca.colormap as cm
from folium import plugins

In [ ]:
# A base map
m_1 = folium.Map(location=[59.9375, 30.308611], tiles=None, zoom_start=9)

# Add a step ColorMap
linear_cm = cm.LinearColormap(["#FFFFF0", "#F7CB15", "#F61005", "#140200"])
step_cm = linear_cm.to_step(8).scale(0, 8000)

# Draw a ColorMap (legend) on a map
step_cm.caption = "Number of CCTV"
m_1.add_child(step_cm)

popup = {}
for v in versions:
    popup[v] = folium.GeoJsonPopup(
        fields=["District", "Number_of_CCTV", "Increase_str"],
        aliases=["District", "Number of CCTV", "Increase"],
        localize=True,
        labels=True,
        style="background-color: #F7CB15;",
    )

tooltip = {}
for v in versions:
    tooltip[v] = folium.GeoJsonTooltip(
        fields=["District", "Number_of_CCTV", "Increase_str"],
        aliases=["District", "Number of CCTV", "Increase"],
        localize=True,
        sticky=True,
        labels=False,
        style="""
            background-color: #F0EFEF;
            border: 1px solid black;
            border-radius: 3px;
            box-shadow: 3px;
        """,
        max_width=800,
)

# The layers with districts
districts_l = {}
for v in versions:
    districts_l[v] = folium.GeoJson(
        versions[v],
        style_function=lambda x: {
            "fillColor": step_cm(x["properties"]["Number_of_CCTV"]),
            "fillOpacity": 0.72,
            "color": "black",
            "weight": 0.75,
        },
        overlay=False,
        name="CCTV",
        tooltip=tooltip[v],
        popup=popup[v],
    )

#   Organize layers
# Main group
fg = folium.FeatureGroup(control=False)
m_1.add_child(fg)

# We need a subgroup for each year to make it easy to change layers
subgroup = {}
year = 2023
for v in reversed(versions):
    subgroup[v] = folium.plugins.FeatureGroupSubGroup(fg, str(year), overlay=False)
    subgroup[v].add_child(folium.TileLayer('cartodbpositronnolabels', name=str(year)))
    subgroup[v].add_child(districts_l[v])
    m_1.add_child(subgroup[v])
    year -= 1

# Add a layer without districts
subgroup["Base"] = folium.plugins.FeatureGroupSubGroup(fg, "Basemap", overlay=False)
subgroup["Base"].add_child(folium.TileLayer('cartodbpositronnolabels', name="Basemap"))
m_1.add_child(subgroup["Base"])

folium.LayerControl(collapsed=False, position='bottomleft').add_to(m_1)

m_1